# Background

There are two parts to our reaction control system, the reaction wheels and the magnetorquer

The magnetorquer will maintain spacecraft orientation relative to the sun, and slowly desaturate the reaction wheels after short-time scale maneuvers. It needs to be strong enough to offset any extrinsic torque on the spacecraft. The magnetorquer can only apply torque on the two axes orthogonal to the earth's magnetic field.

The reaction wheels will provide precise control of pointing and angular velocity over short timescales, and will allow the spacecraft to rotate around the axis of the earth's magnetic field

# Extrinsic Torque

How strong is the extrinsic torque on the spacecraft?

Possible sources include atmospheric drag, solar radiation pressure, thermal radiation (others?)

It might be safe to assume that atmospheric drag is the most significant source of extrinsic torque. From the code below, it seems like torque due to atmospheric drag will be on the order of 1e-4 Nm

In [1]:
import math as m
import matplotlib.pyplot as plt
import numpy as np

# all units kilogram-meter-second
altitude = 330e3 # 200km is lower bound seen on nanosats.eu. 330 is orbit of ISS
rad_earth = 6367e3
u = 4*10**14 # standard gravitational parameter 
a = rad_earth + altitude
v = m.sqrt(u/a)
Cd = 2.2 # from http://digitalcommons.usu.edu/cgi/viewcontent.cgi?article=1144&context=smallsat
area = 0.2*0.3 # planetary labs-style 3u cubesat w/ solar panels extended and facing into the wind
lever_arm = 0.15 # half the length of a 3u cubesat

# altitude (km), density (kg/m^3) during low, avg, high solar activity
# from http://www.braeunig.us/space/atmos.htm
msise_90 = np.array([
[0,     1.17e+00,   1.17e+00,   1.16e+00],
[40,    4.07e-03,   4.07e-03,   4.04e-03],
[80,    1.69e-05,   1.68e-05,   1.68e-05],
[120,   1.70e-08,   1.80e-08,   2.34e-08],
[160,   9.65e-10,   1.18e-09,   2.23e-09],
[200,   1.75e-10,   2.91e-10,   8.28e-10],
[240,   4.31e-11,   9.91e-11,   4.08e-10],
[300,   7.22e-12,   2.58e-11,   1.71e-10],
[400,   5.68e-13,   3.89e-12,   5.04e-11],
[500,   6.03e-14,   7.30e-13,   1.70e-11],
[600,   1.03e-14,   1.56e-13,   6.20e-12],
[700,   3.58e-15,   3.91e-14,   2.38e-12],
[900,   1.18e-15,   5.46e-15,   4.03e-13],
])

p = np.interp(altitude, 1000*msise_90[:,0], msise_90[:,2])

print('altitude = {:.3g} km'.format(altitude/1000))
print('ballistic coefficient = {:.3g}'.format(Cd))
print('area = {:.3g} m^2'.format(area))
print('atmospheric pressure = {:.3g} kg/m^3'.format(p))
print('orbital velocity = {:.3g} m/s'.format(v))

# from https://www.ll.mit.edu/publications/journal/pdf/vol01_no2/1.2.6.satellitedrag.pdf
Fdrag = (1/2) * Cd * area * p * v**2

print()

#print('Fdrag = {:.1g} kg*m/s^2'.format(Fdrag))
print('Fdrag = {:.1g} grams force'.format(Fdrag*102))
#print('Fdrag = {:.1g} lbf'.format(Fdrag*0.2248))

print('Tdrag = {:.1g} Nm'.format(Fdrag*lever_arm))
print('Tdrag = {:.1g} ft lbf'.format(Fdrag*lever_arm*.7376))

altitude = 330 km
ballistic coefficient = 2.2
area = 0.06 m^2
atmospheric pressure = 1.92e-11 kg/m^3
orbital velocity = 7.73e+03 m/s

Fdrag = 0.008 grams force
Tdrag = 1e-05 Nm
Tdrag = 8e-06 ft lbf


# Moment of Inertia

What's the moment of inertia for the spacecraft?
On the order of 1e0 Nm s^2

From http://www.diracdelta.co.uk/science/source/m/o/moments%20of%20inertia/source.html#.V_a-KbWYqRs,

    Solid Cuboid Moment of Inertia
    mass, m =	3 	kg
    length, a =	0.1 	m
    length, b =	0.1 	m
    length, c =	3 	m
    moment of inertia, I1 =	2.2525 	kgm2
    moment of inertia, I2 =	2.2525 	kgm2
    moment of inertia, I3 =	0.005000000000000001 	kgm2



What's the moment of inertia for a bare motor?
On the order of 1e-6 Nm s^2

    Solid Cylinder Moment of Inertia
    mass, m =	0.01 	kg
    radius, r =	0.022 	m
    length, L =	0.006 	m
    moment of inertia, I1,I2 =	0.0000012399999999999998 	kgm2
    moment of inertia, I3 =	0.00000241

From the cubesat spec, 
        
    3.2.14 The CubeSat center of gravity shall be located within 2 cm from its geometric center in
    the X and Y direction.
    3.2.14.1 The 1U CubeSat center of gravity shall be located within 2 cm from its geometric
    center in the Z direction.


# Rate of Rotation Passing over Ground Station

We don't care about constant rotational velocity but we do care about spinning up from a stationary, sun tracking state to a gently turning, ground station tracking state. We care because we need to store that angular momentum in the reaction wheels.










In [4]:
import math as m

# all units kilogram-meter-second
altitude = 330e3 # lower bound seen on nanosats.eu
rad_earth = 6367e3
u = 4*10**14 # standard gravitational parameter 
a = rad_earth + altitude
v = m.sqrt(u/a)

w = m.degrees(m.asin(v/altitude))

print ("angular rate for tracking = {:.1g} degrees / s".format(w))

angular rate for tracking = 2 degrees / s
